In [1]:
from memory_profiler import memory_usage
import time
import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_gradient
from fmm.surface import scale_surface

# Adaptoctree
import adaptoctree.morton as morton
import adaptoctree.tree as tree

In [2]:
# Compile test experiment to cache numba functions
e = Fmm('test')
e.run()
laplace_p2p_serial(e.sources, e.targets, e.source_densities)
laplace_gradient(e.sources, e.targets, e.source_densities)

array([[ 283.3415 ,  602.8579 ,  448.8763 ],
       [ 728.319  ,  836.0533 ,  150.27887],
       [ 731.808  ,  421.88794,  366.041  ],
       ...,
       [-464.09085, -511.3172 , -427.3405 ],
       [-305.128  , -636.4044 , -557.71857],
       [-272.34122, -617.55475,  -90.45561]], dtype=float32)

In [50]:
# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test
! rm C4E4.hdf5 && fmm generate-test-data -c C4E4 && fmm compute-operators -c C4E4
! rm C5E5.hdf5 && fmm generate-test-data -c C5E5 && fmm compute-operators -c C5E5
! rm C6E6.hdf5 && fmm generate-test-data -c C6E6 && fmm compute-operators -c C6E6
! rm C7E7.hdf5 && fmm generate-test-data -c C7E7 && fmm compute-operators -c C7E7

Generating random sources & targets
Computing operators
Computing octree
Computing Inner Surface of Order 4
Computing Outer Surface of Order 4
Computing Inverse of Check To Equivalent Gram Matrix
Computing M2M & L2L Operators
Computed (1/8) M2M/L2L operators
Computed (2/8) M2M/L2L operators
Computed (3/8) M2M/L2L operators
Computed (4/8) M2M/L2L operators
Computed (5/8) M2M/L2L operators
Computed (6/8) M2M/L2L operators
Computed (7/8) M2M/L2L operators
Computed (8/8) M2M/L2L operators
Computed operators for (1/4) M2L Levels
Computed operators for (2/4) M2L Levels
Computed operators for (3/4) M2L Levels
Computed operators for (4/4) M2L Levels
Total time elapsed 0 minutes and 12 seconds
Generating random sources & targets
Computing operators
Computing octree
Computing Inner Surface of Order 5
Computing Outer Surface of Order 5
Computing Inverse of Check To Equivalent Gram Matrix
Computing M2M & L2L Operators
Computed (1/8) M2M/L2L operators
Computed (2/8) M2M/L2L operators
Computed (3/8)

In [51]:
# Discretisation order of check surface
Cvec = [4, 5, 6, 7]

# Discretisation order of equivalent surface
Evec = [4, 5, 6, 7]

# Load experimental data into PyExaFMM
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

In [52]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'C{p}E{p}.dat')
    exafmmvec.append(fmm)
    
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

# Runtimes

In [53]:
# Evaluate PyExaFMM experiments
nruns = 7
pytvec = [[] for _ in range(nruns)]
for i in range(nruns):    
    for e in pyfmmvec:
        s = time.time()
        e.run()
        pytvec[i].append(time.time()-s)
        
pytvec = np.array(pytvec)

In [54]:
# Evaluate ExaFMM-T experiments
nruns = 7
extvec = [[] for _ in range(nruns)]
for i in range(nruns):
    for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
        s = time.time()
        laplace.evaluate(tree, fmm)
        extvec[i].append(time.time()-s)
extvec = np.array(extvec)

In [55]:
pytvec.mean(axis=0)

array([ 13.18985489,  64.77526096,  71.32551016, 130.75818545])

In [56]:
pytvec.std(axis=0)

array([0.20564043, 1.36779165, 3.73098496, 2.33841284])

In [57]:
extvec.mean(axis=0)

array([0.40737987, 1.46008154, 1.66453586, 2.11161297])

In [58]:
extvec.std(axis=0)

array([0.04801554, 0.04118234, 0.04325972, 0.05655091])

# Memory

In [59]:
pyfmmvec[-1].nleaves

32768

In [60]:
pymemvec = []

for e in pyfmmvec:
    pymemvec.append(max(memory_usage(e.run)))

In [61]:
pymemvec

[4929.77734375, 4929.83203125, 4929.83203125, 4929.83203125]

In [62]:
exmemvec = []

for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    def f(): laplace.evaluate(tree, fmm)
    exmemvec.append(max(memory_usage(f)))

In [63]:
exmemvec

[2979.82421875, 3321.06640625, 3553.87890625, 3875.046875]

# Accuracy

In [13]:
direct = laplace_p2p_serial(
    pyfmmvec[0].sources, 
    pyfmmvec[0].targets, 
    pyfmmvec[0].source_densities
)

In [14]:
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

for e in pyfmmvec:
    e.run()

In [17]:
pyfmmvec[-1].nleaves

1888